<img align="center" width="180" src="noktos.jpg">

<h1><center>Revenue Optimization with Dynamic Pricing</center></h1> 

# Revenue Optimization with Dynamic Pricing
### Notebook by [Marvin Nahmias](http://www.noktos.com), original work by M.Tavora
### Table of contents

1. [Dynamic Pricing](#Dynamic-Pricing)
1. [Problem Statement](#Problem-Statement)
2. [Code](#Code)

### Import AI Libraries from Python
<br>

In [12]:
import pandas as pd
import numpy as np 
import psycopg2, scipy, os, calendar, warnings, pickle, math
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
%matplotlib inline
import chart_studio
import chart_studio.plotly as py
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) 
warnings.filterwarnings('ignore')

## Dynamic Pricing
[[go back to the top]](#Table-of-contents)

From [this](https://www.hospitalitynet.org/opinion/4045046.html) source:

> Dynamic pricing means that a hotel will change its room rates daily or even within a day if up-to-the-minute market information reveals the need for adjustments. It is based on the recognition that the right rate to charge for a room night is what the customer is able and willing to pay.

In other words, dynamic pricing is the strategy of adjusting prices as a function of a multitude of factors, such as room availability, demand, occupancy and others. 

Hotels and airline companies have practiced quantity-based RM (Revenue Management) for some time. RM may includes among other things:
- Offering rooms initially at a lower rate
- When the number of bookings surpasses the quantity allocated to the less expensive rooms, which we will call segment $A$, rates change to values corresponding to segment $B$ and so on. The size of the segments is kept fixed but its prices vary with time (naturally).

## Goal
[[go back to the top]](#Table-of-contents)

The goal of this notebook is to describe a hotel revenue management (RM) model that can be implemented in practice.


Note that this is a dynamic chart depending on external factors (such as availability, occupancy, weather, market conditions and macroeconomics).

## Mathematical Formulation
[[go back to the top]](#Table-of-contents)

This is a constrained optimization problem. According to [Aziz et al]([this article](https://www.sciencedirect.com/science/article/pii/S1110866511000375#s0030)):
> To formulate this problem as a price control tactic, we need to change the decision variables to be the prices
set every day.

The objective function is:

$$\sum\limits_{i = 1}^s {\sum\limits_{j = 1}^n {{p_{ij}}} {O_{ij}}}$$

where:
- $j$: night index
- $p_j$ is the price of the some room segment for night $j$
- $O_j$ is the (projected) occupancy for night j at price $p_j$. 
- $n$ is the number of nights 

The decision variable of this optimization problem is the price $p_j$ of night $j$. The constraint is:

$$O = O_{\rm{nominal}}\bigg( \frac{p}{p_{\rm{nominal}}} \bigg)^{e} $$

where $O_{\rm{nominal}}$ is the forecasted occupancy for a given night based on the rate $p_{\rm{nominal}}$ i.e.

$$O_{\rm{nominal}} = f(p_{\rm{nominal}},...),$$

where $p_{\rm{nominal}}$ is the nominal price of the hotel (usually the average historical price).

We fix the base rate to a constant $r$ and the elasticity to another constant $e$. This choice of elasticity was based on [this article](https://www.sciencedirect.com/science/article/pii/S1110866511000375#s0030)

If the hotel has $N_{i}$ rooms in segment i (with $i=A,B,C$ and $D$ i.e. 4 segments in total), the objective function is subject to the following constraints:
- $O_j \leq C_j$: the occupancy can't be greater than the number of available rooms $C_j$
- $p_j \ge 0$: prices cannot be negative


Dynamic pricing involves re-running the forecast of the optimal price per night which was set ahead of time and update the values of $O_j$ based on new data (on a daily/weekly basis). Price levels are set based on current occupancy (as mentioned before, the levels increase as the number of available rooms in the hotel decrease).

This simulated hotel will have four segments. To solve for the optimal price, we must find how many rooms are available and their corresponding price segment. 

## Predicting Occupancy
[[go back to the top]](#Table-of-contents)

We will consider a hotel with four different types of suite $A, B, C$ and $D$ and we will generate predictions for rates of suites type $A$ (making predictions for the others is a trivial extension). Let us suppose that there are $N_A=80$ suites in segment $A$. The room price $p_A$ will be set to $550/night in the high season.

As previously stated, the aim is to find, based on this baseline prediction, the optimal revenue maximizing prices.

In the first figure below, the predicted occupancy is plotted. It shows a weekly seasonality and a (weak) trend upward. Weekend peaks are also present as expected. The dashed line indicates the capacity of rooms of type $A$.

Note that this forecast must be updated frequently using reservation/cancellation data. 

As expected, keeping $p_A$ fixed at $550mxn is a suboptimal strategy for reasons such as:
- Weekdays have lower occupancy and therefore lowering rates would likely increase revenue
- The revenue at the weekend could benefit from higher prices

## Code
[[go back to the top]](#Table-of-contents)

The following function will simulate the occupancy level for a period of time which we will denote by $N$. The parameters are:
- `nb_days`: Length of the forecast period (in days)
- period: Length of the seasonal component. Typically 7 days for weekly fluctuations.
- dc_level: Baseline occupancy 
- noise_level: This number corresponds to the standard deviation of the normal distribution.
- amplitude: amplitude of the periodic signal
- phase: phase shift of the signal 
- trend: slope of the linear trend term (units of rooms/night).

### Simulating occupancy (demand) for the next $N$ days

We will write an auxiliar function that simulated occupancy curves:

$${O_t}{\rm{ =  }}{O_{{\rm{baseline}}}}{\rm{  +  }}Tt{\rm{  +  }}A{\rm{cos}}\left( {\frac{{{\rm{2}}\pi }}{T}t{\rm{  +  }}\phi } \right){\rm{ +  }}\sigma$$



In [13]:
def simulate_demand(nb_days, period=7.0, dc_level=68.0, noise_level=5.0,
                    amplitude=3.0, phase=2.0*math.pi/7.0, trend=0.06):
    
    t = np.linspace(1,nb_days,nb_days)
    noise = noise_level*np.random.normal(size=len(t))

    demand = dc_level + trend*t + amplitude*np.cos(2.0*math.pi*t/period + phase) + noise

    return demand

For $T=90$ days:

In [14]:
date_start, date_end = '2019-07-01', '2019-09-30'
nb_days = (pd.to_datetime(date_end) - pd.to_datetime(date_start)).days

forecasted_demand = simulate_demand(nb_days, 
                                    dc_level=68,
                                    noise_level=1.0, 
                                    amplitude=5)

forecasted_demand = pd.DataFrame(forecasted_demand,
                                 index=pd.date_range(start=date_start,
                                                     end=date_end,
                                                     closed='left'),
                                 columns=['occupancy'])

Plotting the occupancy forecastL

In [15]:
occupancy = [go.Scatter(x=forecasted_demand.index, y=forecasted_demand['occupancy'],
                        name='Forecast of noktos Suite-A occupancy for the next 90 days')]


layout_occ = go.Layout(title='Forecast of noktos Suite-A occupancy for the next 90 days',
                       xaxis=  {'title':'Day'},
                       yaxis=  {'title':'Number of Rooms Occupied'},
                       shapes=[{'type':'line',
                                'x0':'2019-07-01',
                                'x1':'2019-09-30',
                                'y0':80.0, 'y1':80.0,
                                'line': {'color': 'rgb(50, 171, 96)', 
                                         'width': 4, 'dash':'dash'}}])

fig = go.Figure(data=occupancy, layout=layout_occ)
iplot(fig, filename='occ_ts')

We now need the relationship between price and demand. We choose the elasticity $e =−2$ hence the demand or occupancy function becomes:

$$O(p) = O_{\rm{nominal}}\bigg( \frac{p_{\rm{nominal}}}{p} \bigg)^{2} $$

In [16]:
# Let's assume a demand price elasticity function:

def demand_price_elasticity(price, nominal_demand, elasticity=-2.0, nominal_price=550.0):
    
    return nominal_demand * ( price / nominal_price ) ** (elasticity)

The optimal prices for different capacity levels must obey the following constraints:
- $O_j \leq C_j$: the occupancy can't be greater than the number of available rooms $C_j$
- $p_j \ge 0$: prices cannot be negative

The objective function and constraints are given below.

In [17]:
import scipy.optimize as optimize


def objective(p_t, nominal_demand=np.array([50,40,30,20]),
              elasticity=-2.0, nominal_price=550.0):
    return (-1.0 * np.sum( p_t * demand_price_elasticity(p_t, nominal_demand=nominal_demand,
                                                        elasticity=elasticity,
                                                        nominal_price=nominal_price) ))/100

def constraint_1(p_t):
    """ This constraint ensures that the prices are positive.
    """
    return p_t


def constraint_2(p_t, capacity=20, forecasted_demand=35.0,
                 elasticity=-2.0, nominal_price=550.0):
    return capacity - demand_price_elasticity(p_t, nominal_demand=forecasted_demand,
                                                        elasticity=elasticity,
                                                        nominal_price=nominal_price)

Now I will select four overlapping capacity segments and minimize using the SLSQP  technique.

In [18]:
capacities = [20.0, 40.0, 60.0, 80.0]

optimization_results = {}
for capacity in capacities:

    nominal_price = 550.0
    nominal_demand = forecasted_demand['occupancy'].values
    elasticity = -2.0
    p_start = 500.0 * np.ones(len(nominal_demand))
    bounds = tuple((10.0, 2000.0) for p in p_start)

    constraints = ({'type': 'ineq', 'fun':  lambda x:  constraint_1(x)},
               {'type': 'ineq', 'fun':  lambda x, capacity=capacity,
                                           forecasted_demand=nominal_demand,
                                           elasticity=elasticity,
                                           nominal_price=nominal_price: constraint_2(x,capacity=capacity,
                                                                                     forecasted_demand=nominal_demand,
                                                                                     elasticity=elasticity,
                                                                                     nominal_price=nominal_price)})

    opt_results = optimize.minimize(objective, p_start, args=(nominal_demand,
                                                              elasticity,
                                                              nominal_price),
                                    method='SLSQP', bounds=bounds,
                                    constraints=constraints)

    optimization_results[capacity] = opt_results

In [19]:
time_array = np.linspace(1,len(nominal_demand),len(nominal_demand))
rate_df = pd.DataFrame(index=time_array)

for capacity in optimization_results.keys():
    rate_df = pd.concat([rate_df,
                         pd.DataFrame(optimization_results[capacity]['x'],
                                      columns=['{}'.format(capacity)],
                                      index=time_array)],
                        axis=1)

rate_df.index.name = 'Day'
datelist = pd.date_range(start=date_start, end=date_end, closed='left').tolist()
rate_df.index = [ x.date() for x in datelist]

### Prices versus capacity levels


The table and the plot show prices per night and capacity levels. 

The table below contains rates in terms of number of rooms still available e.g.
- Column `Capacity left: 80.0` corresponds to the case where there are all 80 rooms left to book 
- Column `Capacity left: 20.0` represents the case where there are only 20 rooms left to book. 

The prices and capacity move in oposite directions as expected.

In [20]:
rate_df_to_show = rate_df.copy()
rate_df_to_show = rate_df_to_show[np.sort(np.asarray(rate_df_to_show.columns))]
rate_df_to_show.columns = ['Capacity left : {}'.format(x) for x in rate_df_to_show.columns]
for col in rate_df_to_show.columns:
    rate_df_to_show[col] = rate_df_to_show[col].apply(lambda x: round(x,2))
dow_map = { 6:'Sun', 0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat'}
rate_df_to_show['date'] = rate_df_to_show.index
rate_df_to_show['dow'] = rate_df_to_show['date'].apply(lambda x: dow_map[x.weekday()])
rate_df_to_show['date'] = rate_df_to_show.apply(lambda row: row['dow']+" "+str(row['date']),axis=1)
rate_df_to_show.index = rate_df_to_show['date'].values
rate_df_to_show.drop(['date','dow'],axis=1,inplace=True)
rate_df_to_show.head(10)

,Capacity left : 20.0,Capacity left : 40.0,Capacity left : 60.0,Capacity left : 80.0
Mon 2019-07-01,997.65,705.44,575.99,498.82
Tue 2019-07-02,985.09,696.56,568.74,492.55
Wed 2019-07-03,982.60,694.80,567.31,491.30
Thu 2019-07-04,1012.21,715.74,584.40,506.11
Fri 2019-07-05,1032.80,730.30,596.29,516.40
Sat 2019-07-06,1051.21,743.32,606.92,525.60
Sun 2019-07-07,1036.70,733.06,598.54,518.35
Mon 2019-07-08,1011.30,715.10,583.87,505.65
Tue 2019-07-09,988.95,699.29,570.97,494.48
Wed 2019-07-10,982.15,694.49,567.05,491.08


### Plotting the room rate time series.

In [21]:
price_levels = [go.Scatter(x=rate_df_to_show.head(7).index,
                           y=rate_df_to_show.head(7)['Capacity left : 20.0'],
                           name='Capacity Remaining : 20 rooms'),
                go.Scatter(x=rate_df_to_show.head(7).index,
                           y=rate_df_to_show.head(7)['Capacity left : 40.0'],
                           name='Capacity Remaining : 40 rooms'),
                go.Scatter(x=rate_df_to_show.head(7).index,
                           y=rate_df_to_show.head(7)['Capacity left : 60.0'],
                           name='Capacity Remaining : 60 rooms'),
                go.Scatter(x=rate_df_to_show.head(7).index,
                           y=rate_df_to_show.head(7)['Capacity left : 80.0'],
                           name='Capacity Remaining : 80 rooms')]

layout_prices = go.Layout(title='Rate vs Reservation Date and Current Capacity Levels',
                       xaxis={'title':'Day'}, yaxis={'title':'Rate ($)'})

fig = go.Figure(data=price_levels, layout=layout_prices)
iplot(fig, filename='price_levels_ts')

Now we save results in a comma separated file.

In [22]:
rate_df.to_csv('projected-rates.csv')